# Populate Replay Buffer

In [ ]:
# !pip install minerl anyrl

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import json
import gym

from collections import deque

import numpy as np

from anyrl.rollouts import replay

import pickle

In [ ]:
import sys
sys.path.append("../functions")

from Utils import save_data, load_data
from Buffer import populate_buffer, add_transition
from DataHelper import parse_load_data

In [ ]:
replay_buffer = replay.PrioritizedReplayBuffer(500000, alpha=0.4, beta=0.6, epsilon=0.001)

In [ ]:
environment = 'Treechop'
folder = 'new-resources'

In [ ]:
# file_path = "/content/drive/My Drive/minerl-resources/datasets/parsed_data_treechop.sav"
# combos_path = "/content/drive/My Drive/minerl-resources/combos.sav"
# unique_places_path = "/content/drive/My Drive/minerl-resources/unique_placements.sav"
# unique_actions_path = "/content/drive/My Drive/minerl-resources/unique_actions_treechop.sav"

file_path = "../{0}/parsed_data.sav".format(folder)
combos_path = "../{0}/action_combos_{1}.sav".format(folder, environment.lower())
unique_places_path = "../{0}/unique_placements.sav".format(folder)
unique_actions_path = "../{0}/unique_actions_{1}.sav".format(folder, environment.lower())

In [ ]:
combos = load_data(combos_path)
unique_actions = load_data(unique_actions_path)
unique_places = load_data(unique_places_path)
parsed_data = load_data(file_path)

In [ ]:
replay_buffer = populate_buffer(parsed_data, replay_buffer, combos, environment)

In [ ]:
save_data('../{0}/buffer_actions_{1}.sav'.format(folder, environment.lower()), replay_buffer)